## 第1回 FR Frontier ：ファッション画像における洋服の「色」分類

* [コンテスト詳細 ビッグデータ活用ならオプトDSL DeepAnalytics](https://deepanalytics.jp/compe/36)

## preprocess

In [1]:
import numpy as np
import pandas as pd

In [7]:
import os, sys
current_dir = os.getcwd()
HOME_DIR = current_dir
DATA_GIVEN_DIR = HOME_DIR+"/data/given/"
DATA_RESIZED_DIR = HOME_DIR+"/data/resized/"
DATA_MYDATA_DIR = HOME_DIR+"/data/processed/"

In [2]:
width = 100
height = 100
channel = 3

In [3]:
%pwd

'/output'

In [4]:
%ls $DATA_GIVEN_DIR

category_master.tsv  sample_code.zip    test/     train/     train_master.tsv
sample_code/         sample_submit.csv  test.zip  train.zip


### Resize & Transparent

In [15]:
os.makedirs(DATA_RESIZED_DIR)
os.makedirs(DATA_RESIZED_DIR+"train/")
os.makedirs(DATA_RESIZED_DIR+"test/")

In [16]:
from PIL import Image

def resize_and_trans_img(src, dst, width, height):
    img = Image.open(src, 'r')

    # リサイズ。サイズは幅と高さをtupleで指定
    resized = img.resize((width, height))

    # 同じサイズの画像を作成
    trans = Image.new('RGBA', resized.size, (0, 0, 0, 0))

    for x in range(width):
        for y in range(height):
            pixel = resized.getpixel( (x, y) )
        
            # 白なら処理しない
            if pixel[0] == 255 and pixel[1] == 255 and pixel[2] == 255:
                continue
        
            # 白以外なら、用意した画像にピクセルを書き込み
            trans.putpixel( (x, y), pixel )
    # リサイズ後の画像を保存
    trans.save(dst, 'JPEG', optimize=True)

In [17]:
for i in range(0, 12399):
    src = DATA_GIVEN_DIR + 'train/' + 'train_%i.jpg'%i
    dst = DATA_RESIZED_DIR + "train/" + 'train_%i.jpg'%i
    resize_and_trans_img(src, dst, width, height)

In [18]:
for i in range(0, 9801):
    src = DATA_GIVEN_DIR + 'test/' + 'test_%i.jpg'%i
    dst = DATA_RESIZED_DIR + "test/" + 'test_%i.jpg'%i
    resize_and_trans_img(src, dst, width, height)

### train/valid/test directory 作成

In [19]:
## train/valid/test directory 作成
category_master = pd.read_table(DATA_GIVEN_DIR+'category_master.tsv', decimal='\t')
category_len = len(category_master['category_id'])

os.makedirs(DATA_MYDATA_DIR+"train/")
for i in range(0, category_len):
    os.makedirs(DATA_MYDATA_DIR+"train/%i"%i)
os.makedirs(DATA_MYDATA_DIR+"valid/")
for i in range(0, category_len):
    os.makedirs(DATA_MYDATA_DIR+"valid/%i"%i)

In [20]:
train_master = pd.read_table(DATA_GIVEN_DIR+'train_master.tsv', decimal='\t')
train_master.head()

,file_name,category_id
0,train_0.jpg,21
1,train_1.jpg,22
2,train_2.jpg,11
3,train_3.jpg,18
4,train_4.jpg,21


In [21]:
## copy image to categorised directory
import shutil
for i, row in train_master.iterrows():
    src = DATA_GIVEN_DIR + "train/" + row['file_name']
    dst = DATA_MYDATA_DIR + "train/%i/%s"%(row['category_id'], row['file_name'])
    shutil.copy(src, dst)

In [22]:
shutil.copytree(DATA_GIVEN_DIR + "test/", DATA_MYDATA_DIR + "test/unknown")

'/home/tsu-nera/repo/deepanalytics/uniqlo/data/processed/test/unknown'

In [23]:
%cd $DATA_MYDATA_DIR/train

/home/tsu-nera/repo/deepanalytics/uniqlo/data/processed/train


In [24]:
import glob
g = glob.glob('*/*.jpg')
shuf = np.random.permutation(g)
for i in range(3000): 
    shutil.move(shuf[i], '../valid/' + shuf[i])

### create sample data(Optional)

In [84]:
%cd $DATA_MYDATA_DIR

/home/tsu-nera/repo/deepanalytics/uniqlo/data/processed


In [91]:
os.makedirs("sample/")
os.makedirs("sample/train/")
os.makedirs("sample/valid/")
DATA_SAMPLE_DIR = DATA_MYDATA_DIR + "sample/"

In [86]:
for i in range(0, category_len):
    os.makedirs("sample/train/%i"%i)
for i in range(0, category_len):
    os.makedirs("sample/valid/%i"%i)

In [87]:
%cd $DATA_MYDATA_DIR/train

/home/tsu-nera/repo/deepanalytics/uniqlo/data/processed/train


In [ ]:
import glob
import shutil
g = glob.glob('*/*.jpg')
shuf = np.random.permutation(g)
for i in range(700): shutil.copy(shuf[i], '../sample/train/' + shuf[i])

In [89]:
%cd $DATA_MYDATA_DIR/valid

/home/tsu-nera/repo/deepanalytics/uniqlo/data/processed/valid


In [90]:
g = glob.glob('*/*.jpg')
shuf = np.random.permutation(g)
for i in range(300): shutil.copy(shuf[i], '../sample/valid/' + shuf[i])

## training

In [4]:
import keras
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, Conv2D, BatchNormalization, MaxPooling2D, Convolution2D
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [5]:
batch_size=64

In [75]:
def get_batches(path, gen=ImageDataGenerator(rescale=1./255), shuffle=True, batch_size=8, 
                class_mode='categorical'):
    return gen.flow_from_directory(path, target_size=(width,height),
                                   class_mode=class_mode, 
                                   shuffle=shuffle,
                                   batch_size=batch_size)

In [76]:
gen_t = ImageDataGenerator(rotation_range=15, width_shift_range=0.1, rescale=1./255,
                           height_shift_range=0.1, zoom_range=0.1, horizontal_flip=True)

In [77]:
train_path = '/input/train' # for floyd
#train_path = DATA_MYDATA_DIR + 'train/'
valid_path = '/input/valid' # for floyd
#valid_path = DATA_MYDATA_DIR + 'valid/'
batches = get_batches(train_path, 
                      gen_t, batch_size=batch_size)
# NB: We don't want to augment or shuffle the validation set
val_batches = get_batches(valid_path, shuffle=False, 
                          batch_size=batch_size)

Found 9399 images belonging to 24 classes.
Found 3000 images belonging to 24 classes.


In [78]:
model = Sequential([
  Conv2D(16, (3, 3), padding='same', strides=(2, 2), input_shape=(width, height, 3), activation='relu'),
  MaxPooling2D(pool_size=(3, 3)),
  Dropout(0.2),

  Conv2D(32, (3, 3), padding='same', activation='relu'),
  MaxPooling2D(pool_size=(3, 3)),
  Dropout(0.2),

  Conv2D(64, (3, 3), padding='same', activation='relu'),
  MaxPooling2D(pool_size=(2, 2)),
  Dropout(0.2),

  Flatten(),
  Dense(128, activation='tanh'),
  Dropout(0.3),
  Dense(24, activation='softmax'),
])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 50, 50, 16)        448       
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 16, 16, 16)        0         
_________________________________________________________________
dropout_38 (Dropout)         (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 16, 16, 32)        4640      
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 5, 5, 32)          0         
_________________________________________________________________
dropout_39 (Dropout)         (None, 5, 5, 32)          0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 5, 5, 64)          18496     
__________

In [17]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(width,height,3)),
        Conv2D(32,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Conv2D(64,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Conv2D(128,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Flatten(),
        Dense(200, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(200, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(24, activation='softmax')
    ])

Exception: Input 0 is incompatible with layer convolution3d_8: expected ndim=5, found ndim=4

In [9]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_1 (BatchNorma (None, 224, 224, 3)   896         batchnormalization_input_1[0][0] 
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 222, 222, 32)  896         batchnormalization_1[0][0]       
____________________________________________________________________________________________________
batchnormalization_2 (BatchNorma (None, 222, 222, 32)  888         convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 111, 111, 32)  0           batchnormalization_2[0][0]       
___________________________________________________________________________________________

In [79]:
model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [80]:
batches.samples//batch_size

146

In [81]:
model.fit_generator(batches, batches.samples//batch_size, epochs=3, 
                    validation_data=val_batches, 
                    validation_steps=val_batches.samples//batch_size)

Epoch 1/3
  9/146 [>.............................] - ETA: 133s - loss: 3.0487 - acc: 0.0990

KeyboardInterrupt: 

In [136]:
# model.save_weights('2.h5')

In [ ]:
# model.load_weights('1.h5')

## Predict

In [44]:
test_path = '/input/test'
#test_path = DATA_MYDATA_DIR + 'test'
test_batches = get_batches(test_path, shuffle=False, 
                           batch_size=batch_size)

Found 9801 images belonging to 1 classes.


In [56]:
test_features = model.predict_generator(test_batches, test_batches.samples//batch_size + 1)

In [57]:
test_labels = np.argmax(test_features, axis=1)
test_labels[:5]

array([14,  4, 17, 15, 20])

In [58]:
filenames = test_batches.filenames
filenames = [a[8:] for a in filenames]

In [59]:
filenames[:5]

['test_9405.jpg',
 'test_5841.jpg',
 'test_2295.jpg',
 'test_1566.jpg',
 'test_936.jpg']

In [60]:
import re
def get_file_no(filename):
    r = re.compile("[0-9]+")
    no = r.search(filename)
    return int(no.group())

In [61]:
len(filenames), len(test_labels)

(9801, 9801)

In [62]:
subm = np.stack([filenames,test_labels], axis=1)
submission = pd.DataFrame(subm, columns=['filename','category_id'])
submission['file_no'] = submission['filename'].apply(get_file_no)
submission = submission.sort_values(by="file_no", ascending=True)
submission = submission.drop("file_no", axis=1)

In [63]:
submission_file_name = 'submission.csv'
#np.savetxt(submission_file_name, subm, delimiter=',')
submission.to_csv(submission_file_name, index=False, header=None)

In [64]:
from IPython.display import FileLink
FileLink(submission_file_name)

/output/submission.csv